In [1]:
import os
import torch
# import wandb

import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import GroupKFold
folds = GroupKFold(n_splits=5)

In [3]:
import pandas as pd
train_label_image = pd.read_csv('./siim-covid19-detection/train_image_level.csv')
train_label_study = pd.read_csv('./siim-covid19-detection/train_study_level.csv')

In [4]:
train_label_study

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1
...,...,...,...,...,...
6049,ffcb4630f46f_study,0,1,0,0
6050,ffe4d6e8fbb0_study,0,1,0,0
6051,ffe94fcb14fa_study,0,1,0,0
6052,ffebf1ef4a9c_study,0,1,0,0


In [5]:
study_id = set()
boxes_study_id= set()
uniqued_train_label_image = pd.DataFrame(columns = train_label_image.columns)

for idx,image_id,boxes,label,StudyInstanceUID in train_label_image.itertuples():
    # break
    if StudyInstanceUID not in study_id:
        study_id.add(StudyInstanceUID)
        # check if opacity in label
        if label.find('none') == -1:
            boxes_study_id.add(StudyInstanceUID)
        # print(uniqued_train_label_image.columns)
        # print(train_label_image.iloc[idx])
        uniqued_train_label_image = uniqued_train_label_image.append(train_label_image.iloc[idx])
        # print(uniqued_train_label_image.columns)
        # break
    else:
        if StudyInstanceUID in boxes_study_id:
            continue
        elif label.find('none') == -1:
            boxes_study_id.add(StudyInstanceUID)
            uniqued_train_label_image.loc[uniqued_train_label_image['StudyInstanceUID'] == StudyInstanceUID] = [image_id,boxes,label,StudyInstanceUID]

uniqued_train_label_image = uniqued_train_label_image.reset_index(drop=True)

In [6]:
train_label_study['StudyInstanceUID'] = train_label_study['id'].apply(lambda x: x.replace('_study',''))
train_df = pd.merge(uniqued_train_label_image,train_label_study,on='StudyInstanceUID')
train_df.rename(columns={'id_x':'id_image'},inplace=True)
train_df.drop(['id_y'],axis=1,inplace=True)

In [7]:
train_df['id_image'] = train_df['id_image'].apply(lambda x: x.replace('_image',''))
train_df['label_2class'] = train_df['label'].apply(lambda x: 1 if x.find('opacity') != -1 else 0)
train_df


,id_image,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,1,0,0,0
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0,1,0,0
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0,0,0,1
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,0,1,0,0
...,...,...,...,...,...,...,...,...
6049,ffcc6edd9445,NaN,none 1 0 0 1 1,7e6c68462e06,1,0,0,0
6050,ffd91a2c4ca0,NaN,none 1 0 0 1 1,8332bdaddb6e,1,0,0,0
6051,ffd9b6cf2961,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,0,1,0,0
6052,ffdc682f7680,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d,0,1,0,0


In [8]:
train_df['filepath'] = f'./tmp_{512}/train/'+train_df['id_image']+'.png'


In [9]:
from sklearn.model_selection import GroupKFold
folds = GroupKFold(n_splits=5)
train_df['fold_id'] = -1
for fold_id,(train_idx,valid_idx) in enumerate(folds.split(train_df,groups=train_df.StudyInstanceUID.tolist())):
    train_df.loc[valid_idx,'fold_id'] = fold_id

In [10]:
import os

os.makedirs(train_path,exist_ok=True)
os.makedirs(valid_path,exist_ok=True)

NameError: name 'train_path' is not defined

In [11]:
from tqdm.auto import tqdm
from shutil import copyfile
for fold_id in range(5):
    train_path = f'data_2classfy/fold{fold_id}/train'
    valid_path = f'data_2classfy/fold{fold_id}/test'
    for class_name in ['none','opacity']:
        os.makedirs(os.path.join(train_path,class_name),exist_ok=True)
        os.makedirs(os.path.join(valid_path,class_name),exist_ok=True)
    train_fold_df = train_df[train_df.fold_id != fold_id]
    valid_fold_df = train_df[train_df.fold_id == fold_id]
    for idx,row in tqdm(train_fold_df.iterrows(),total=len(train_fold_df)):
        if row['label_2class'] == 0:
            copyfile(row.filepath,os.path.join(train_path,'none',row.id_image+'.png'))
        else:
            copyfile(row.filepath,os.path.join(train_path,'opacity',row.id_image+'.png'))
    for idx,row in tqdm(valid_fold_df.iterrows(),total=len(valid_fold_df)):
        if row['label_2class'] == 0:
            copyfile(row.filepath,os.path.join(valid_path,'none',row.id_image+'.png'))
        else:
            copyfile(row.filepath,os.path.join(valid_path,'opacity',row.id_image+'.png'))

  0%|          | 0/4843 [00:00<?, ?it/s]

  0%|          | 0/1211 [00:00<?, ?it/s]

  0%|          | 0/4843 [00:00<?, ?it/s]

  0%|          | 0/1211 [00:00<?, ?it/s]

  0%|          | 0/4843 [00:00<?, ?it/s]

  0%|          | 0/1211 [00:00<?, ?it/s]

  0%|          | 0/4843 [00:00<?, ?it/s]

  0%|          | 0/1211 [00:00<?, ?it/s]

  0%|          | 0/4844 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

In [32]:
# generante data.yaml
import yaml
    
for fold_id in range(5):
    data_yaml = dict(
        train = f'data_classfy/fold{fold_id}/train/images',
        val = f'data_classfy/fold{fold_id}/valid/images',
        nc = 4,
        names = ['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']
    )
    with open(f'data_classfy/fold{fold_id}/data.yaml','w') as f:
        yaml.dump(data_yaml,f)

In [14]:
df = pd.concat([train_df, valid_df]).reset_index(drop=True)

In [15]:
print(f'Size of dataset: {len(df)}, training images: {len(train_df)}. validation images: {len(valid_df)}')


Size of dataset: 6334, training images: 5067. validation images: 1267


In [16]:
os.makedirs(f'data_{IMG_SIZE}/train/images', exist_ok=True)
os.makedirs(f'data_{IMG_SIZE}/valid/images', exist_ok=True)


In [17]:
os.makedirs(f'data_{IMG_SIZE}/train/labels', exist_ok=True)
os.makedirs(f'data_{IMG_SIZE}/valid/labels', exist_ok=True)


In [18]:
for i in tqdm(range(len(df))):
    row = df.loc[i]
    if row.split == 'train':
        copyfile(row.path, f'data_{IMG_SIZE}/train/images/{row.id}.png')
    else:
        copyfile(row.path, f'data_{IMG_SIZE}/valid/images/{row.id}.png')

  0%|          | 0/6334 [00:00<?, ?it/s]

In [19]:
os.listdir(f'./tmp_{IMG_SIZE}/train')[:5]

['7183445f4d28.png',
 '1ea96fd9dd7c.png',
 '273722673feb.png',
 '3566e20a178e.png',
 '8d7cfdfc8bf2.png']

In [20]:
row.path

'./tmp_640/train/d7a2083fec46.png'

In [21]:
import yaml

data_yaml = dict(
    train = f'/home/outerform/github-repo/DeepLearningCourse-Project/data_{IMG_SIZE}/train/images',
    val = f'/home/outerform/github-repo/DeepLearningCourse-Project/data_{IMG_SIZE}/valid/images',
    nc = 2,
    names = ['none', 'opacity']
)

with open(f'data_{IMG_SIZE}.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    


In [22]:
def get_bbox(row):
    bboxes = []
    bbox = []
    labels  = row.label.split(' ')
    bboxes = []
    for i in range(0,len(labels), 6):
        # print(labels[i+2:i+6])
        bboxes.append(list(map(float,labels[i+2:i+6])))
    return np.array(bboxes)

        
        

In [23]:
def scale(row,bboxes):
    scalex = IMG_SIZE/row.dim1
    scaley = IMG_SIZE/row.dim0
    scaled_bboxes = np.zeros_like(bboxes)
    scaled_bboxes[:,[0,2]] = bboxes[:,[0,2]]*scalex
    scaled_bboxes[:,[1,3]] = bboxes[:,[1,3]]*scaley
    return scaled_bboxes

In [24]:
def yolo_format(bboxes):
    x = ((bboxes[:,0]+bboxes[:,2])/2)/IMG_SIZE
    y = ((bboxes[:,1]+bboxes[:,3])/2)/IMG_SIZE
    width = (bboxes[:,2]-bboxes[:,0])/IMG_SIZE
    height = (bboxes[:,3]-bboxes[:,1])/IMG_SIZE
    return np.column_stack([x,y,width,height]).astype(np.float32).reshape(-1,4)

In [25]:
def convert_to_yolo(row):
    bboxes = get_bbox(row)
    scaled_bboxes = scale(row,bboxes)
    yolo_bboxes = yolo_format(scaled_bboxes)
    # image_id = row.id
    # label = [0]*len(yolo_bboxes)
    return yolo_bboxes

In [26]:
for i in tqdm(range(len(df))):
    row = df.loc[i]
    # print(row.split)
    label = row.image_level
    if row.split == 'train':
        label_path = f'data_{IMG_SIZE}/train/labels/{row.id}.txt'
    else:
        label_path = f'data_{IMG_SIZE}/valid/labels/{row.id}.txt'
    
    if label == 'opacity':
        yoloboxes = convert_to_yolo(row)
        with open(label_path, 'w') as f:
            for bbox in yoloboxes:
                f.write(f'1 {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n')


  0%|          | 0/6334 [00:00<?, ?it/s]

In [27]:
x = np.array([1,2])
y= np.array([1,3])